<a href="https://colab.research.google.com/github/fernando-Silva-developer/Classificador-de-imagem-para-supermercado/blob/main/C%C3%B3pia_de_trabalho_Ia2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importando as bibliotecas

In [ ]:
import urllib.request
import os
from PIL import Image,ImageStat
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision
import torch
import torch.nn as nn
from torch.utils.data import  DataLoader
import torchvision.transforms as transforms
import torchsummary
from torch import optim
import sklearn.metrics as metrics
from google.colab import drive
drive.mount('/content/drive')

Modelo utilizado

In [ ]:

class LeNet2(nn.Module):
    def __init__(self):
        super(LeNet2,self).__init__()
        self.backbone = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5,stride=1,padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2,padding=0),
            nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2,padding=0),
            nn.Flatten()
        )
        self.classifier = nn.Sequential(
            nn.Linear(1296,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,2),
        )


    def forward(self,x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x


In [ ]:
model = LeNet2()

In [ ]:
model.to('cuda:0')

In [ ]:
model.classifier = nn.Sequential(nn.Linear(1296,2))

In [ ]:
model

In [ ]:
import torch
import torchsummary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

torchsummary.summary(model, (3, 50, 50), device=str(device))


In [ ]:
torchsummary.summary(model,(3,50,50))

Importando um modelo do pytorch

In [ ]:
model = torchvision.models.efficientnet_b0(weights = torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1)

In [ ]:
model

In [ ]:
model.classifier[1] = nn.Linear(1280,2)

In [ ]:
model

In [ ]:
for layer in model.children():
    print(layer)

In [ ]:
main_dir = '/content/drive/My Drive/frutasdef'

In [ ]:
os.chdir(main_dir)

In [ ]:
import torch
from torchvision import transforms, datasets



transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

image_dataset = datasets.ImageFolder(root=main_dir, transform=transform)

In [ ]:
print(image_dataset)

In [ ]:
dir(image_dataset)

In [ ]:
image_dataset.classes

In [ ]:
image_dataset.class_to_idx

In [ ]:
image_dataset.targets[0]

In [ ]:
image_dataset[0]

In [ ]:
dataset = image_dataset

In [ ]:
len(dataset)

In [ ]:
np.unique(dataset.targets,return_counts=True)

In [ ]:
def image_loader(img):
    return Image.open(img).convert("RGB")

In [ ]:
dataset = torchvision.datasets.DatasetFolder('./',loader = image_loader,extensions='jpg',transform=transform )

In [ ]:
dataset.targets = torch.tensor(dataset.targets)

In [ ]:
ds = image_dataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
ds

In [ ]:
ds.targets = np.array(ds.targets)

Dividindo o dataset em treino, teste e validação

In [ ]:
bs = 64
train_idx, temp_idx = train_test_split(np.arange(len(ds)),test_size=0.3,shuffle=True,stratify=ds.targets)
valid_idx, test_idx = train_test_split(temp_idx,test_size=0.5,shuffle=True,stratify=ds.targets[temp_idx])

train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
valid_sampler = torch.utils.data.SubsetRandomSampler(valid_idx)
test_sampler  = torch.utils.data.SubsetRandomSampler(test_idx)

dl_train = torch.utils.data.DataLoader(ds,batch_size=bs,sampler=train_sampler)
dl_valid = torch.utils.data.DataLoader(ds,batch_size=bs,sampler=valid_sampler)
dl_test  = torch.utils.data.DataLoader(ds,batch_size=bs,sampler=test_sampler)

In [ ]:
x,y = next(iter(dl_train))

In [ ]:
x.shape,y.shape

In [ ]:
device = torch.device('cuda:0')

In [ ]:
device

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.get_device_properties(0)

In [ ]:
x.to(device)

In [ ]:
torch.cuda.get_device_properties(0)

In [ ]:
model.to(device)

In [ ]:
loss_train = []
loss_eval  = []

In [ ]:
patience_time = 15

In [ ]:
criterion = nn.CrossEntropyLoss()
opt = optim.SGD(model.parameters(),lr=0.01)

###TREINANDO O MODELO


Treinei ele apenas 29 vezes porque demora muito pra rodar mais vezes com o "patient_time" = 15, então limitei o número de iterações para conseguir rodar em menos de 1 dia.

In [ ]:
epochs = 100
count = 0
stop = False
epoch = 0
lowest_loss_eval = 10000
last_best_result = 0
count = 0
while (not stop):
    model.train()
    lloss = []
    for x,y in dl_train:
        x = x.to(device)
        y = y.to(device)
        pred = model(x)
        closs = criterion(pred,y)
        closs.backward()
        opt.step()
        opt.zero_grad()
        lloss.append(closs.item())
    loss_train.append(np.mean(lloss))
    lloss = []
    model.eval()
    lres = []
    ytrue = []
    with torch.no_grad():
        for data,y in dl_valid:
            data = data.to(device)

            pred = model(data)
            closs = criterion(pred.cpu(),y)
            lloss.append(closs.item())
            res  = pred.argmax(dim=1).cpu().tolist()
            lres += res
            ytrue += y
    avg_loss_eval = np.mean(lloss)
    loss_eval.append(avg_loss_eval)
    if avg_loss_eval < lowest_loss_eval:
        lowest_loss_eval = avg_loss_eval
        last_best_result = 0
        print("Best model found! saving...")
        count += 1
        print(count)
        actual_state = {'optim':opt.state_dict(),'model':model.state_dict(),'epoch':epoch,'loss_train':loss_train,'loss_eval':loss_eval}
        torch.save(actual_state,'best_model.pth')
    last_best_result += 1
    if last_best_result > patience_time:
        stop = True
    print("epoch %d loss_train %4.3f loss_eval %4.3f last_best %d"%(epoch,loss_train[-1],loss_eval[-1],last_best_result))
    epoch += 1

In [ ]:
import torch

In [ ]:
recover = torch.load('best_model.pth')
actual_state = {'optim':opt.state_dict(),'model':model.state_dict(),'epoch':epoch}

In [ ]:
opt.load_state_dict(recover['optim'])
model.load_state_dict(recover['model'])
loss_train = recover['loss_train']
loss_eval = recover['loss_eval']
epoch = recover['epoch']

In [ ]:
l = []
for layer in model.children():
    l.append(layer)

In [ ]:
model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
x = torch.randn(1, 3, 448, 448).to(device)
output = model(x)

In [ ]:
x = torch.randn(1,3,448,448).to('cuda:0')
model(x)

In [ ]:
import matplotlib.pyplot as plt


fig = plt.figure()
plt.plot(loss_train[1:])
plt.plot(loss_eval[1:])
plt.xlabel('Epochs')
plt.show()

In [ ]:
import sklearn.metrics as metrics

In [ ]:
model.to(device)

In [ ]:
model.eval()
lres = []
ytrue = []
with torch.no_grad():
    for data,target in dl_test:
        data = data.to(device)
        pred = model(data)
        res  = pred.argmax(dim=1).cpu().tolist()
        lres += res
        ytrue += target

In [ ]:
metrics.confusion_matrix(ytrue,lres)

In [ ]:
metrics.accuracy_score(ytrue,lres)

In [ ]:
metrics.precision_score(ytrue,lres)

In [ ]:
metrics.recall_score(ytrue,lres)

In [ ]:
metrics.f1_score(ytrue,lres)

In [ ]:
print(metrics.classification_report(ytrue,lres))

###IMPORTANDO UMA IMAGEM PARA TESTAR O MODELO

In [ ]:
image_loader = lambda x: Image.open(x).convert("RGB")

In [ ]:
img2 = image_loader('./macajpg/maca.jpg')

In [ ]:
img = image_loader('./bananajpn/IMG_4912.jpg')

In [ ]:
x = transform(img)

In [ ]:
y = transform(img2)

In [ ]:
x = x.view(1,3,224,224)

In [ ]:
y = y.view(1,3, 224, 224)

In [ ]:
x = x.to(device)

In [ ]:
y = y.to(device)

In [ ]:
pred = model(x)

In [ ]:
pred2 = model(y)

In [ ]:
pred

In [ ]:
pred2

###FAZENDO UM TESTE NO MODELO

In [ ]:
previsao = model(x)
classe_predita = torch.argmax(previsao, dim=1)
if classe_predita.item() == 1:
  print("A imagem é uma maçã.")
else:
  print("A imagem é uma banana.")

###CRIANDO O BOT DO TELEGRAM

In [ ]:
!pip install telebot

In [ ]:
import telebot
from PIL import Image
import io
import torch

# Insira o token do seu bot
TOKEN = "7833060106:AAEq7kIBS6NhW78bfuX1YlbhQ3Ugt-OjiUg"
bot = telebot.TeleBot(TOKEN)

# Variável global para armazenar a imagem
stored_image = None

@bot.message_handler(commands=['start'])
def start(message):
    bot.reply_to(message, "Envie-me uma imagem para que eu a armazene.")

@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    global stored_image

    # Obtém o ID do arquivo da foto em alta resolução
    file_id = message.photo[-1].file_id

    # Baixa o arquivo da foto
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    # Armazena a imagem em uma variável usando Pillow
    stored_image = Image.open(io.BytesIO(downloaded_file))

    # Transformação e classificação
    x = transform(stored_image).view(1, 3, 224, 224).to(device)
    previsao = model(x)
    classe_predita = torch.argmax(previsao, dim=1)

    if classe_predita.item() == 1:
        bot.reply_to(message, "Maçã!")
    elif classe_predita.item() == 0:
        bot.reply_to(message, "Banana!")
    else:
        bot.reply_to(message, "ERRO")

@bot.message_handler(commands=['get_image'])
def get_stored_image(message):
    if stored_image:
        bot.reply_to(message, "A imagem foi armazenada.")
    else:
        bot.reply_to(message, "Nenhuma imagem foi armazenada ainda.")

bot.polling()